In [1]:
import torch

# Check if CUDA is available
if torch.cuda.is_available():
    print("CUDA is available.")
else:
    print("CUDA is not available. Check if your GPU drivers are properly installed.")


CUDA is available.


In [2]:
# !pip install datasets>=2.6.1
# !pip install transformers
# !pip install librosa
# !pip install jiwer
# !pip install evaluate>=0.30
# #!pip install gradio
# !pip install -U accelerate

#from 20.5.2024
# !pip install audiomentations[extras]
# !pip install mutagen
# !pip install srt

In [3]:
import os
os.chdir('/app/')


import librosa
import random
import numpy as np
import IPython.display as ipd
import pickle
import pandas as pd
from datasets import Dataset
from datasets import Audio
from torch.utils.data import ConcatDataset
from transformers import WhisperProcessor
import mutagen.mp3
from tqdm import tqdm
import json
from audiomentations import Compose, AddGaussianNoise, TimeStretch, PitchShift, Shift, RoomSimulator
import srt
import re
from IPython.display import clear_output
from transformers import WhisperForConditionalGeneration
from huggingface_hub import login



#our libraries
from global_variables.training_vars import *
from global_variables.folders import *
from parashat_hashavua_dataset import *
from nikud_and_teamim import just_teamim,remove_nikud


/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# load the token from txt file
with open("./tokens/HF_token.txt", "r") as f:
    HF_TOKEN = f.read().strip() # strip() removes the trailing "\n" if it exists
login(token=HF_TOKEN)


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [5]:
processor = WhisperProcessor.from_pretrained("openai/whisper-" + BASE_MODEL_VERSION, language="hebrew", task="transcribe")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
# # Check how many tokens needed for each nikud/taam
# chars = ['֑', '֒', '֓', '֔', '֕', '֖', '֗', '֘', '֙', '֚', '֛', '֜', '֝', '֞', '֟', '֠', '֡', '֢', '֣', '֤', '֥', '֦', '֧', '֨', '֩', '֪', '֫', '֬', '֭', '֮', '֯', 'ְ', 'ֱ', 'ֲ', 'ֳ', 'ִ', 'ֵ', 'ֶ', 'ַ', 'ָ', 'ֹ', 'ֺ', 'ֻ', 'ּ', 'ֽ', '־', 'ֿ', '׀', 'ׁ', 'ׂ', '׃', 'ׄ', 'ׅ', '׆', 'ׇ']
# for c in chars:
#     print(f"{c}: {len(processor.tokenizer.encode(c))-5}")


In [7]:
tokens_added = (len(processor.tokenizer.encode('֟'))==6) # check if the tokens were already added
if ADDTOKENS and not tokens_added: # add the tokens if they weren't already added
    
    teamim = ['֑', '֒', '֓', '֔', '֕', '֖', '֗', '֘', '֙', '֚', '֛', '֜', '֝', '֞', '֟', '֠', '֡', '֢', '֣', '֤', '֥', '֦', '֧', '֨', '֩', '֪', '֫', '֬', '֭', '֮', 'ֽ']
    if JUST_TEAMIM:
        new_tokens = [BASE_CHAR + c for c in teamim] # add the base char to the teamim (e.g. א֑)
    elif NIKUD:
        new_tokens = ['֑', '֒', '֓', '֔', '֕', '֖', '֗', '֘', '֙', '֚', '֛', '֜', '֝', '֞', '֟', '֠', '֡', '֢', '֣', '֤', '֥', '֦', '֧', '֨', '֩', '֪', '֫', '֬', '֭', '֮', '֯', 'ְ', 'ֱ', 'ֲ', 'ֳ', 'ִ', 'ֵ', 'ֶ', 'ַ', 'ָ', 'ֹ', 'ֺ', 'ֻ', 'ּ', 'ֽ', '־', 'ֿ', '׀', 'ׁ', 'ׂ', '׃', 'ׄ', 'ׅ', '׆', 'ׇ']
    else:
        new_tokens = teamim
    
    processor.tokenizer.add_tokens(new_tokens)

In [8]:
if COMBINE_DATA:
    train_data1 = parashat_hashavua_dataset(new_data = True, few_data=FASTTEST, train =True ,validation=False, random=RANDOM, num_of_words_in_sample=4, nusachim=NUSACHIM, augment=AUGMENT, processor=processor)
    train_data2 = parashat_hashavua_dataset(new_data = False, train =True ,validation=False, random=RANDOM, num_of_words_in_sample=13, augment=AUGMENT, prob_for_num_of_parts = [0.05, 0.05, 0.05, 0.05, 0.1, 0.15, 0.15, 0.2, 0.2], processor=processor)
    train_data = ConcatDataset([train_data1, train_data2])
else:
    if NEWDATA == "other":
        train_data = parashat_hashavua_dataset(new_data = NEWDATA, few_data=FASTTEST, train =True ,validation=False, random=RANDOM, num_of_words_in_sample=1, nusachim=NUSACHIM, augment=AUGMENT, processor=processor)
    elif NEWDATA:
        train_data = parashat_hashavua_dataset(new_data = True, few_data=FASTTEST, train =True ,validation=False, random=RANDOM, num_of_words_in_sample=4, nusachim=NUSACHIM, augment=AUGMENT, processor=processor)

    else:
        train_data = parashat_hashavua_dataset(new_data = False, train =True ,validation=False, random=RANDOM, num_of_words_in_sample=13, augment=AUGMENT, prob_for_num_of_parts = [0.05, 0.05, 0.05, 0.05, 0.1, 0.15, 0.15, 0.2, 0.2], processor=processor)
        
        
print(len(train_data))


dict_keys(['text', 'maroko', 'yerushalmi', 'ashkenazi', 'bavly'])


Loading ashkenazi nusach (1/4):   0%|          | 0/500 [00:00<?, ?it/s]

Loading ashkenazi nusach (1/4):  88%|████████▊ | 442/500 [00:01<00:00, 360.73it/s]

the text doesn't have nikud:  סוף חמישי ותחילת ששי למנהג האשכנזים


Loading ashkenazi nusach (1/4): 100%|██████████| 500/500 [00:01<00:00, 276.02it/s]


...

Num of missing files in yerushalmi nusach:  5
dict_keys(['text', 'maroko', 'yerushalmi', 'ashkenazi', 'bavly'])


Loading bavly nusach (4/4):  88%|████████▊ | 442/500 [00:01<00:00, 255.65it/s]

the text doesn't have nikud:  סוף חמישי ותחילת ששי למנהג האשכנזים
the audio is not mp3 or the length is not legal:  audio_files_new_data/bavly_01-bereshit_6_10510_1.mp3
the audio is not mp3 or the length is not legal:  audio_files_new_data/bavly_01-bereshit_6_10510_2.mp3
the audio is not mp3 or the length is not legal:  audio_files_new_data/bavly_01-bereshit_6_10510_3.mp3
the audio is not mp3 or the length is not legal:  audio_files_new_data/bavly_01-bereshit_6_10510_4.mp3
the audio is not mp3 or the length is not legal:  audio_files_new_data/bavly_01-bereshit_6_10510_5.mp3


Loading bavly nusach (4/4): 100%|██████████| 500/500 [00:02<00:00, 240.83it/s]

Num of missing files in bavly nusach:  6
1983


In [27]:
train_data = parashat_hashavua_dataset(new_data = False, train =True ,validation=False, random=True, num_of_words_in_sample=13, augment=AUGMENT, prob_for_num_of_parts = [0.05, 0.05, 0.05, 0.05, 0.1, 0.15, 0.15, 0.2, 0.2], processor=processor)

100%|██████████| 357/357 [00:58<00:00,  6.12it/s]


success_count:  333
fail_count:  24
-25: 1
-9: 1
-8: 1
-7: 3
-5: 1
-4: 2
-2: 4
-1: 8
1: 2
2: 1
diff_list:  [(-25, 'Beshalach-4.txt'), (-9, 'Haazinu-1.txt'), (-8, 'Haazinu-2.txt'), (-7, 'Haazinu-5.txt'), (-7, 'Haazinu-6.txt'), (-7, 'Haazinu-3.txt'), (-5, 'Haazinu-4.txt'), (-4, 'Yitro-6.txt'), (-4, 'Vaethanan-4.txt'), (-2, 'Vayishlach-6.txt'), (-2, 'Shmini-6.txt'), (-2, 'Behaalotcha-5.txt'), (-2, 'Nasso-6.txt'), (-1, 'Devarim-5.txt'), (-1, 'Mishpatim-1.txt'), (-1, 'Behar-5.txt'), (-1, 'Shemot-1.txt'), (-1, 'LechLecha-4.txt'), (-1, 'Emor-1.txt'), (-1, 'Bamidbar-3.txt'), (-1, 'Vayeshev-6.txt'), (1, 'Tetzaveh-3.txt'), (1, 'KiTisa-4.txt'), (2, 'Tzav-3.txt')]


In [38]:
processor.tokenizer.decode(train_data[0]['labels'])

'<|startoftranscript|><|he|><|transcribe|><|notimestamps|>הול֣יד את־ וכ֥ה תדבֽר׃ מעש֔ך ובכ֖ל א֛יש על־ ובֽרכך֙ והט֖וב קֽדש־ ה֚מה והי֗ה<|endoftext|>'

In [10]:
# check the data of Pocket Torah
# train_data.check_failed_files_of_old_data()

In [11]:
# play random from the train data
# train_data.print_and_play_word_by_index(random.randint(0, len(train_data.data)))


#validation of the data
# train_data.check_the_data() 
# remove the sample index:
# train_data.remove_word_by_index(32487)


In [12]:
if COMBINE_DATA:
    val_data1 = parashat_hashavua_dataset(new_data = True, few_data=FASTTEST, train=False ,validation=True,  num_of_words_in_sample=4, nusachim=NUSACHIM, processor=processor)
    val_data2 = parashat_hashavua_dataset(new_data = False, train=False ,validation=True, num_of_words_in_sample=13, random=RANDOM, processor=processor)
    val_data = ConcatDataset([val_data1, val_data2])
    
else:
    if NEWDATA == "other":
        val_data = parashat_hashavua_dataset(new_data = NEWDATA, few_data=FASTTEST, train=False ,validation=True,  num_of_words_in_sample=1, nusachim=NUSACHIM, processor=processor)
    elif NEWDATA:
        val_data = parashat_hashavua_dataset(new_data = True, few_data=FASTTEST, train=False ,validation=True,  num_of_words_in_sample=4, nusachim=NUSACHIM, processor=processor)
    else:
        val_data = parashat_hashavua_dataset(new_data = False, train=False ,validation=True, num_of_words_in_sample=13, random=RANDOM, processor=processor)
    

print(len(val_data))

dict_keys(['text', 'maroko', 'yerushalmi', 'ashkenazi', 'bavly'])


Loading ashkenazi nusach (1/4):   4%|▍         | 108/2766 [00:00<00:12, 215.48it/s]

the audio is not mp3 or the length is not legal:  audio_files_new_data/ashkenazi_29-ahare-mot_5_31802_2.mp3


Loading ashkenazi nusach (1/4):   9%|▉         | 246/2766 [00:01<00:11, 222.48it/s]

the audio is not mp3 or the length is not legal:  audio_files_new_data/ashkenazi_18-mishpatim_7_22327_1.mp3


...

the audio is not mp3 or the length is not legal:  audio_files_new_data/bavly_07-vayese_3_12920_2.mp3
the audio is not mp3 or the length is not legal:  audio_files_new_data/bavly_07-vayese_5_13037_3.mp3
the audio is not mp3 or the length is not legal:  audio_files_new_data/bavly_07-vayese_2_12914_1.mp3


Loading bavly nusach (4/4): 100%|██████████| 2766/2766 [00:10<00:00, 260.30it/s]


the audio is not mp3 or the length is not legal:  audio_files_new_data/bavly_14-vaera_6_20914_7.mp3
the audio is not mp3 or the length is not legal:  audio_files_new_data/bavly_14-vaera_6_20915_2.mp3
Num of missing files in bavly nusach:  159


100%|██████████| 21/21 [00:03<00:00,  5.27it/s]

success_count:  21
fail_count:  0
diff_list:  []
3055


In [13]:
print("train new data length: ", len(train_data1))
print("train old data length: ", len(train_data2))

print("val new data length: ", len(val_data1))
print("val old data length: ", len(val_data2))

train new data length:  85010
train old data length:  69820
val new data length:  2692
val old data length:  363


In [14]:
from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [15]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

In [16]:
import concurrent.futures
import evaluate
import time
import cantilLocations_evaluation


# # possible metrics : "wer", "cer", "bleu", "rouge", "sacrebleu", "sari":
# # 1. `wer`: Word Error Rate.
# # 2. `cer`: Character Error Rate.
# # 3. `bleu`: Bilingual Evaluation Understudy.
# # 4. `rouge`: Recall-Oriented Understudy for Gisting Evaluation.
# # 5. `sacrebleu`: A standardized BLEU score implementation for more consistent machine translation evaluation.
# # 6. `sari`: System Agnostic Refinement Index. 

WER_CALCULATOR = evaluate.load("wer")
def compute_metrics(pred):
    eval_list = cantilLocations_evaluation.calculate_precision_recall_f1_for_string_list_with_method_list
    pred_ids = pred.predictions
    label_ids = pred.label_ids
    
    # method to calculate the metrics(method can be "Exact", "Letter_Shift", "Word_Level", "Word_Shift")
    methods = ["Exact", "Letter_Shift", "Word_Level", "Word_Shift"]

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = processor.tokenizer.pad_token_id
    
    start_time = time.time()
    
    # we do not want to group tokens when computing the metrics
    pred_str = processor.tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = processor.tokenizer.batch_decode(label_ids, skip_special_tokens=True)
    
    decode_time = time.time() - start_time
    
    # evaluate the metrics
    results = eval_list(pred_str, label_str, methods)
    
    
    
    # compute the average of each metric
    avg = {}
    for i in range(4):
        avg["avg_precision_" + methods[i]] = np.mean(results[i][0])
        avg["avg_recall_" + methods[i]] = np.mean(results[i][1])
        avg["avg_f1_" + methods[i]] = np.mean(results[i][2])
    
    precision_list_exact = results[methods.index("Exact")][0]
    recall_list_exact = results[methods.index("Exact")][1]
    f1_list_exact = results[methods.index("Exact")][2]
    
    # compute the median
    precision_median_exact = np.median(precision_list_exact)
    recall_median_exact = np.median(recall_list_exact)
    f1_median_exact = np.median(f1_list_exact)
    
    
    # max and min:
    precision_max_exact = np.max(precision_list_exact)
    recall_max_exact = np.max(recall_list_exact)
    f1_max_exact = np.max(f1_list_exact)
    best_index = np.argmax(f1_list_exact)
    
    f1_min = [0, 0, 0, 0]
    recall_min = [0, 0, 0, 0]
    precision_min = [0, 0, 0, 0]
    
    for i in range(4):
        precision_min[i] = np.min(results[i][0])
        recall_min[i] = np.min(results[i][1])
        f1_min[i] = np.min(results[i][2])
    
    worst_index = [np.argmin(results[i][2]) for i in range(4)] 
    
    
    
    start_time = time.time()
    # WER
    wer = 100 * WER_CALCULATOR.compute(predictions=pred_str, references=label_str)
    
    wer_time = time.time() - start_time
    
    best_pred = pred_str[best_index]
    best_label = label_str[best_index]
    worst_pred = [pred_str[worst_index[i]] for i in range(4)]
    worst_label = [label_str[worst_index[i]] for i in range(4)]
    
    # print
    # best:
    print(f"best f1 for {methods[0]}: {f1_max_exact}\nbest pred: {best_pred}\nbest label: {best_label}\n")
    
    # worst (the worst for each method):
    for i in range(4):
        print(f"worst f1 for {methods[i]}: {f1_min[i]}\nworst pred: {worst_pred[i]}\nworst label: {worst_label[i]}\n")
    
    
    
    print("Time taken for each part:")
    print(f"Decode calculation: {decode_time} seconds")
    print(f"WER calculation: {wer_time} seconds")
    
    # matric_dict = {"wer": wer, "precision": precision_avg, "recall": recall_avg, "f1": f1_avg, "precision_median": precision_median, "recall_median": recall_median, "f1_median": f1_median, "precision_max": precision_max, "recall_max": recall_max, "f1_max": f1_max, "precision_min": precision_min, "recall_min": recall_min, "f1_min": f1_min}
    
    # create the matric_dict with the metrics
    matric_dict = {"wer": wer}
    for i in range(4):
        matric_dict["avg_precision_" + methods[i]] = avg["avg_precision_" + methods[i]]
        matric_dict["avg_recall_" + methods[i]] = avg["avg_recall_" + methods[i]]
        matric_dict["avg_f1_" + methods[i]] = avg["avg_f1_" + methods[i]]
    matric_dict["precision_median_exact"] = precision_median_exact
    matric_dict["recall_median_exact"] = recall_median_exact
    matric_dict["f1_median_exact"] = f1_median_exact
    matric_dict["precision_max_exact"] = precision_max_exact
    matric_dict["recall_max_exact"] = recall_max_exact
    matric_dict["f1_max_exact"] = f1_max_exact
    for i in range(4):
        matric_dict["precision_min_" + methods[i]] = precision_min[i]
        matric_dict["recall_min_" + methods[i]] = recall_min[i]
        matric_dict["f1_min_" + methods[i]] = f1_min[i]
    # print(matric_dict)
    return matric_dict

In [17]:
INIT_OUTPUT_LAYER = False

In [18]:
model = WhisperForConditionalGeneration.from_pretrained(BASE_MODEL_NAME, use_cache=False) # we can add "force_download=True" to download the model again

model.generation_config.language = "he" 




if ADDTOKENS:
    model.resize_token_embeddings(len(processor.tokenizer))
    

model.config.forced_decoder_ids = None
model.config.suppress_tokens = []

model.config.decoder_input_ids = None



import torch.nn as nn


# # initialize the last layer of the model:
# if INIT_OUTPUT_LAYER:
#     # Get the proj_out layer
#     proj_out = model.get_output_embeddings()

#     # Reinitialize the weights
#     nn.init.xavier_uniform_(proj_out.weight)

#     # If there's a bias, reinitialize it too
#     if proj_out.bias is not None:
#         nn.init.zeros_(proj_out.bias)




In [19]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir= MODEL_NAME,  # change to a repo name of your choice
    per_device_train_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
    learning_rate=LR, # was 1e-5
    warmup_steps=WARMUP_STEPS, # was 500
    max_steps=MAX_STEPS, # was 4000
    gradient_checkpointing=True,
    gradient_checkpointing_kwargs={'use_reentrant':False}, # I added that because UserWarning: "The default value of use_reentrant will be updated to be False in the future."
    fp16=torch.cuda.is_available(), # I added that because fp16 can't be use on CPU but on cuda
    per_device_eval_batch_size=32,
    predict_with_generate=True,
    eval_strategy="steps", 
    generation_max_length=225, # without this, it crushes the tesorboard
    save_steps=SAVE_STEPS, 
    eval_steps=EVAL_STEPS,   
    logging_steps=1, 
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model= "avg_f1_Exact",# "avg_f1_..." like "avg_f1_Exact"
    greater_is_better=True, # if we use f1 score in eval so greater is better
    push_to_hub=True,
    # I added the dataloader_prefetch_factor to support newer versions of torch (now it must be int and not None. and the default is 2).
    dataloader_prefetch_factor=2, # support newer versions of torch
    dataloader_num_workers=1, # parallelize the data loading
    weight_decay=WEIGHT_DECAY,
    # run_name=RUN_NAME, # It doesn't work
)


In [20]:
# less validation data for testing
val_data = [val_data[i] for i in range(0, len(val_data), 16)] # take every 16th element

In [21]:
from transformers import Seq2SeqTrainer, TrainerCallback


class EvaluateFirstStepCallback(TrainerCallback):
    def on_step_begin(self, args, state, control, model, tokenizer, optimizer, lr_scheduler, train_dataloader, eval_dataloader, **kwargs):
        if state.global_step == 0:
            control.should_evaluate = True



trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=train_data,
    eval_dataset=val_data,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
    callbacks = [EvaluateFirstStepCallback()] if EVALUATE_FIRST_STEP else None
)



max_steps is given, it will override any value given in num_train_epochs


In [22]:
processor.save_pretrained(training_args.output_dir)

[]

In [23]:
def flags_warnings():
    if FASTTEST:
        for i in range(10):
            print("!!!TEST-MODE!!! \t \t to test the code only")

    if not ADDTOKENS:
        print("!!!ADDTOKENS==False!!!")

    if not NEWDATA:
        print("!!!NEWDATA==False!!!")

In [24]:
flags_warnings()

trainer_state = trainer.train()

!!!TEST-MODE!!! 	 	 to test the code only
!!!TEST-MODE!!! 	 	 to test the code only
!!!TEST-MODE!!! 	 	 to test the code only
!!!TEST-MODE!!! 	 	 to test the code only
!!!TEST-MODE!!! 	 	 to test the code only
!!!TEST-MODE!!! 	 	 to test the code only
!!!TEST-MODE!!! 	 	 to test the code only
!!!TEST-MODE!!! 	 	 to test the code only
!!!TEST-MODE!!! 	 	 to test the code only
!!!TEST-MODE!!! 	 	 to test the code only


Step,Training Loss,Validation Loss,Wer,Avg Precision Exact,Avg Recall Exact,Avg F1 Exact,Avg Precision Letter Shift,Avg Recall Letter Shift,Avg F1 Letter Shift,Avg Precision Word Level,Avg Recall Word Level,Avg F1 Word Level,Avg Precision Word Shift,Avg Recall Word Shift,Avg F1 Word Shift,Precision Median Exact,Recall Median Exact,F1 Median Exact,Precision Max Exact,Recall Max Exact,F1 Max Exact,Precision Min Exact,Recall Min Exact,F1 Min Exact,Precision Min Letter Shift,Recall Min Letter Shift,F1 Min Letter Shift,Precision Min Word Level,Recall Min Word Level,F1 Min Word Level,Precision Min Word Shift,Recall Min Word Shift,F1 Min Word Shift
20,6.066600,5.885577,100.000000,0.002481,0.000304,0.000542,0.047388,0.035392,0.037132,0.008411,0.026165,0.012517,0.144453,0.130780,0.131303,0.000000,0.000000,0.000000,0.076923,0.009434,0.016807,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


this sequence of  4  words is too long!
sequence audio length:  32.8620625
sequence text length(in tokens):  48
text:  לֽא־מ֖ות תמתֽון׃ כ֚י יד֣ע אלה֔ים כ֗י ביום֙ אכלכ֣ם ממ֔נו
this sequence of  4  words is too long!
sequence audio length:  32.9549375
sequence text length(in tokens):  46
text:  כ֚י יד֣ע אלה֔ים כ֗י ביום֙ אכלכ֣ם ממ֔נו ונפקח֖ו עֽיניכ֑ם
best f1 for Exact: 0.01680672268907563
best pred:  וה֢֑֡֡֜֨֨֨֨֨֨֨֨֨֨֨֨ר֡ו֡�֡�֡�֡�֡�֡�֡�֡�֡�֡�֡�֡�֡�֡�֡�֡�֡�֡�֡�֡�֡�֡�֡�֡�֡�֡�֡�֡�֡�֡�֡�֡�֡�֡�֡�֡�֡�֡�֡�֡�֡�֡�֡�֡�֡�֡�֡�֡�֡�֡�֡�֡�֡�֡�֡�֡�֡�֡�֡�֡�֡�֡�֡�֡�֡�֡�֡�֡�֡�֡�֡�֡�֡�֡�֡�֡�֡�֡�֡�֡�֡�֡�֡�֡�֡�֡�֡�֡�֡�֡�֡�֡�֡�֡�֡�֡�֡�֡�֡�֡�
best label: וי֨אמר֙ אברה֔ם והֽער֥יך את֖ו הכה֑ן היל֥ך בתֽורת֖י אם־לֽא׃ כֽאש֛ר

worst f1 for Exact: 0.0
worst pred:  גם א�֜ק֧ול֡ול֡ים וחול֑֨֨֨֨֨֨֨֨֨֨ר֨ר֨֨ר֨֨֨ר֨֨ר֨֨ר֨֨֨ר֨֨֨֨֨֨ר֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨֨
worst label: ג֤ם כל־חלי֙ וכל־מכ

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 448, 'suppress_tokens': [], 'begin_suppress_tokens': [220, 50257]}
There were missing keys in the checkpoint model loaded: ['proj_out.weight'].


In [25]:
# model.generation_config.save_pretrained(MODEL_NAME, push_to_hub=True)

In [26]:
kwargs = {
    # "dataset_args": "config: he, split: test",
    "language": "he",
    "model_name": "he-cantillation",
    "finetuned_from": BASE_MODEL_NAME,
    "tasks": "automatic-speech-recognition-cantillation",
    "tags": "hf-asr-leaderboard",
}

In [27]:
# # create a GenerationConfig file
# from transformers import GenerationConfig



In [28]:

trainer.save_model()

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 448, 'suppress_tokens': [], 'begin_suppress_tokens': [220, 50257]}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 448, 'suppress_tokens': [], 'begin_suppress_tokens': [220, 50257]}


In [29]:
trainer.push_to_hub(**kwargs)
# processor.push_to_hub("cantillation" +training_args.output_dir[1:])

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 448, 'suppress_tokens': [], 'begin_suppress_tokens': [220, 50257]}


CommitInfo(commit_url='https://huggingface.co/cantillation/Teamim-tiny_WeightDecay-0.05_Augmented_New-Data_date-10-07-2024_11-18/commit/f9e2b629563f6de444026acfa5aa68aa6996b963', commit_message='End of training', commit_description='', oid='f9e2b629563f6de444026acfa5aa68aa6996b963', pr_url=None, pr_revision=None, pr_num=None)

In [30]:
trainer.lr_scheduler.get_lr()

[7e-06, 7e-06]

In [31]:
processor.tokenizer.decode(train_data[26]["labels"])

'<|startoftranscript|><|he|><|transcribe|><|notimestamps|>אש֖ר מע֣ל לֽרק֑יע וֽיהי־כֽן׃ ויקר֧א אלה֛ים לֽרק֖יע שמ֑ים<|endoftext|>'

In [32]:
# !pip install peft

In [33]:

from datetime import datetime


def log_training_to_markdown_file(training_args, training_loss, epoch, step, validation_loss, f1, recall, precision, filename="training_log.md"):
    # Get the current date and time
    now = datetime.now()

    # Format the date and time as a string
    date_time = now.strftime("%m/%d/%Y, %H:%M:%S")

    with open(filename, 'a') as f:
        f.write(f"| {date_time} | {training_args.output_dir } | {training_args.per_device_train_batch_size} | {training_args.gradient_accumulation_steps} | {training_args.learning_rate} | {training_args.warmup_steps} | {training_args.max_steps} | {training_args.gradient_checkpointing} | {training_args.gradient_checkpointing_kwargs} | {training_args.fp16} | {training_args.eval_strategy} | {training_args.per_device_eval_batch_size} | {training_args.predict_with_generate} | {training_args.generation_max_length} | {training_args.save_steps} | {training_args.eval_steps} | {training_args.logging_steps} | {training_args.report_to} | {training_args.load_best_model_at_end} | {training_args.metric_for_best_model} | {training_args.greater_is_better} | {training_args.push_to_hub} | {training_loss} | {epoch} | {step} | {validation_loss} | {f1} | {recall} | {precision} |\n")

def create_markdown_file_with_headers(filename="./markdown_files/training_log_new.md"):
    with open(filename, 'w') as f:
        f.write("| Date Time | Repo Name | Batch Size | Gradient Accumulation Steps | Learning Rate | Warmup Steps | Max Steps | Gradient Checkpointing | Gradient Checkpointing Kwargs | FP16 | Evaluation Strategy | Eval Batch Size | Predict with Generate | Max Length | Save Steps | Eval Steps | Logging Steps | Report To | Load Best Model at End | Metric for Best Model | Greater is Better | Push to Hub | Training Loss | Epoch | Step | Validation Loss | f1 | recall | precision |\n")
        f.write("|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|----|---|---|\n")

# Create the Markdown file with headers
#create_markdown_file_with_headers()
        

def get_logs_with_step(trainer, step = 1500):
    # Initialize an empty dictionary to store the merged logs
    merged_logs_with_step = {}

    # Iterate over the log history
    for log in trainer.state.log_history:
        # Check if the 'step' key exists in the log and if it equals the provided step
        if 'step' in log and log['step'] == step:
            # If it does, merge the log into the merged_logs_with_step dictionary
            merged_logs_with_step.update(log)

    # Return the merged logs
    return merged_logs_with_step


# Get the training loss
training_loss = trainer_state.training_loss
# Get the step and epoch from the TrainerState
step = trainer.state.global_step
epoch = trainer.state.epoch

# Get the log history at the specified step
history = get_logs_with_step(trainer,training_args.max_steps)
# Get the evaluation details from the log history
validation_loss = history['eval_loss']
f1 = history['eval_avg_f1_Exact']
recall = history['eval_avg_recall_Exact']
precision = history['eval_avg_precision_Exact']

# Log the training details
log_training_to_markdown_file(training_args, training_loss, epoch, step, validation_loss, f1, recall, precision, filename="./markdown_files/training_log_new.md")


In [34]:
#load the markdown file
from IPython.display import Markdown

# Open the file in read mode
with open('./markdown_files/training_log_new.md', 'r') as file:
    # Read the content of the file
    content = file.read()

# Display the content as Markdown
display(Markdown(content))

| Date Time | Repo Name | Batch Size | Gradient Accumulation Steps | Learning Rate | Warmup Steps | Max Steps | Gradient Checkpointing | Gradient Checkpointing Kwargs | FP16 | Evaluation Strategy | Eval Batch Size | Predict with Generate | Max Length | Save Steps | Eval Steps | Logging Steps | Report To | Load Best Model at End | Metric for Best Model | Greater is Better | Push to Hub | Training Loss | Epoch | Step | Validation Loss | f1 | recall | precision |
|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|----|---|---|
| 03/21/2024, 08:40:08 | ./Teamim-AllNusah-whisper-small_Random-True_Mid | 8 | 1 | 1e-05 | 1000 | 100000 | True | {'use_reentrant': False} | True | steps | 32 | True | 225 | 10000 | 10000 | 25 | ['tensorboard'] | True | avg_f1_Exact | True | True | 0.021550073864677687 | 8.0 | 100000 | 0.184482142329216 | 0.8996304230096901 | 0.8996085856138029 | 0.9003176340674156 |
| 03/21/2024, 13:03:45 | ./Teamim-AllNusah-whisper-base_Random-True_Mid | 8 | 1 | 1e-05 | 1000 | 100000 | True | {'use_reentrant': False} | True | steps | 32 | True | 225 | 10000 | 10000 | 25 | ['tensorboard'] | True | avg_f1_Exact | True | True | 0.03545441636119969 | 8.0 | 100000 | 0.20118673145771027 | 0.8617680742203201 | 0.8626021186921445 | 0.8617811978432426 |
| 03/21/2024, 16:41:01 | ./Teamim-AllNusah-whisper-tiny_Random-True_Mid | 8 | 1 | 1e-05 | 1000 | 100000 | True | {'use_reentrant': False} | True | steps | 32 | True | 225 | 10000 | 10000 | 25 | ['tensorboard'] | True | avg_f1_Exact | True | True | 0.04949023559953086 | 8.0 | 100000 | 0.21391823887825012 | 0.8324883831395741 | 0.8338224092406237 | 0.8322131178574591 |
| 03/23/2024, 07:00:56 | ./Teamim-AllNusah-whisper-medium_Random-True_Mid | 8 | 1 | 1e-05 | 1000 | 100000 | True | {'use_reentrant': False} | True | steps | 32 | True | 225 | 10000 | 10000 | 25 | ['tensorboard'] | True | avg_f1_Exact | True | True | 0.016411178880098305 | 8.0 | 100000 | 0.19175758957862854 | 0.9241458540823655 | 0.9248030309610401 | 0.9241210924324863 |
| 05/15/2024, 04:43:26 | ./Teamim-medium_Random-True_OriginalData | 8 | 1 | 1e-05 | 100 | 20000 | True | {'use_reentrant': False} | True | steps | 32 | True | 225 | 10000 | 200 | 25 | ['tensorboard'] | True | avg_f1_Exact | True | True | 0.06837314007226378 | 1.6 | 20000 | 0.32565802335739136 | 0.7752270688239551 | 0.7795095778021163 | 0.7720285537597026 |
| 05/16/2024, 20:40:49 | ./Teamim-medium_Random-True_OriginalData | 8 | 1 | 1e-05 | 100 | 5000 | True | {'use_reentrant': False} | True | steps | 32 | True | 225 | 1000 | 500 | 25 | ['tensorboard'] | True | avg_f1_Exact | True | True | 0.1853820906639099 | 0.4 | 5000 | 0.14098824560642242 | 0.892768311387271 | 0.8960657159022651 | 0.890549476438386 |
| 06/26/2024, 20:15:14 | ./Teamim-large-v2_Random-True_Augmented_date-26-06-2024_10-34-39 | 8 | 1 | 1e-05 | 50 | 8000 | True | {'use_reentrant': False} | True | steps | 32 | True | 225 | 4000 | 1000 | 25 | ['tensorboard'] | True | avg_f1_Exact | True | True | 0.07693093957193196 | 0.64 | 8000 | 0.10415194183588028 | 0.908830758369049 | 0.9087716050227665 | 0.9096437277663242 |
| 06/27/2024, 07:35:51 | ./Teamim-large-v2_Random-True_DropOut-0.3_WeightDecay-1e-05_Augmented_date-26-06-2024_22-25-46 | 8 | 1 | 1e-05 | 50 | 8000 | True | {'use_reentrant': False} | True | steps | 32 | True | 225 | 4000 | 1000 | 25 | ['tensorboard'] | True | avg_f1_Exact | True | True | 0.07779751290939749 | 0.64 | 8000 | 0.10385266691446304 | 0.905166113465824 | 0.9050807123897499 | 0.9059804085487052 |
| 06/27/2024, 10:36:32 | ./Teamim-tiny_Random-True_DropOut-0.1_WeightDecay-1e-06_Augmented_date-27-06-2024_09-48-25 | 8 | 1 | 0.0001 | 50 | 20000 | True | {'use_reentrant': False} | True | steps | 32 | True | 225 | 4000 | 4000 | 25 | ['tensorboard'] | True | avg_f1_Exact | True | True | 0.0803652085084468 | 1.6 | 20000 | 0.16346947848796844 | 0.8548035032054514 | 0.8549944618611862 | 0.8554998107697397 |
| 06/27/2024, 13:27:58 | ./Teamim-tiny_Random-True_DropOut-0.1_WeightDecay-1e-06_Augmented_date-27-06-2024_11-10-07 | 8 | 1 | 0.0001 | 50 | 100000 | True | {'use_reentrant': False} | True | steps | 32 | True | 225 | 10000 | 10000 | 25 | ['tensorboard'] | True | avg_f1_Exact | True | True | 0.025959196008964208 | 8.0 | 100000 | 0.25516340136528015 | 0.8109987026864801 | 0.811303476205186 | 0.8114957652968597 |
| 06/28/2024, 08:39:08 | ./Teamim-large-v2_Random-True_DropOut-0.3_Augmented_date-27-06-2024_20-55-08 | 8 | 1 | 1e-05 | 50 | 10000 | True | {'use_reentrant': False} | True | steps | 32 | True | 225 | 4000 | 1000 | 25 | ['tensorboard'] | True | avg_f1_Exact | True | True | 0.06029476949423552 | 0.8 | 10000 | 0.10647618770599365 | 0.9152141727991959 | 0.915339132943093 | 0.9157900829991397 |
| 06/29/2024, 11:21:42 | ./Teamim-large-v2_DropOut-0.5_Augmented_Combined-Data_date-28-06-2024_16-28 | 8 | 1 | 1e-05 | 50 | 30000 | True | {'use_reentrant': False} | True | steps | 32 | True | 225 | 5000 | 5000 | 25 | ['tensorboard'] | True | avg_f1_Exact | True | True | 0.04124099235981703 | 1.550147263990079 | 30000 | 0.08113238960504532 | 0.9307567418702926 | 0.9306261921216916 | 0.9315657931117158 |
| 07/07/2024, 07:02:58 | ./Teamim-tiny_DropOut-0.5_Augmented_Combined-Data_date-06-07-2024_20-19 | 8 | 1 | 1e-05 | 1000 | 500000 | True | {'use_reentrant': False} | True | steps | 32 | True | 225 | 10000 | 10000 | 25 | ['tensorboard'] | True | avg_f1_Exact | True | True | 0.015838049768279662 | 25.83445282628914 | 500000 | 0.28065910935401917 | 0.8382398748092069 | 0.8396515727745174 | 0.8377787036458046 |
| 07/07/2024, 11:17:43 | ./Teamim-tiny_DropOut-0.5_Augmented_Combined-Data_date-07-07-2024_08-34 | 8 | 1 | 1e-06 | 1000 | 50000 | True | {'use_reentrant': False} | True | steps | 32 | True | 225 | 2000 | 2000 | 25 | ['tensorboard'] | True | avg_f1_Exact | True | True | 4.609999258880615 | 2.583445282628914 | 50000 | 3.9591798782348633 | 0.009378289372690269 | 0.04822695035460992 | 0.005298875867134267 |
| 07/09/2024, 10:06:01 | ./Teamim-tiny_DropOut-0.5_Augmented_Combined-Data_date-09-07-2024_09-42 | 8 | 1 | 1e-05 | 1000 | 5000 | True | {'use_reentrant': False} | True | steps | 32 | True | 225 | 5000 | 5000 | 25 | ['tensorboard'] | True | avg_f1_Exact | True | True | 0.22825482025146485 | 0.2583445282628914 | 5000 | 0.24147212505340576 | 0.6445448712872567 | 0.646263029110693 | 0.6444298169163766 |
| 07/09/2024, 10:37:24 | ./Teamim-tiny_DropOut-0.5_Augmented_Combined-Data_date-09-07-2024_10-17 | 8 | 1 | 1e-05 | 1000 | 5000 | True | {'use_reentrant': False} | True | steps | 32 | True | 225 | 5000 | 5000 | 25 | ['tensorboard'] | True | avg_f1_Exact | True | True | 0.7733024370193482 | 0.2583445282628914 | 5000 | 0.3109682500362396 | 0.5581406054525616 | 0.5585648823768209 | 0.5594666132101971 |
| 07/09/2024, 11:12:56 | ./Teamim-tiny_DropOut-0.5_New-Data_date-09-07-2024_11-11 | 8 | 1 | 1e-05 | 10 | 500 | True | {'use_reentrant': False} | True | steps | 32 | True | 225 | 500 | 500 | 25 | ['tensorboard'] | True | avg_f1_Exact | True | True | 1.0505921382904053 | 2.0161290322580645 | 500 | 1.3115513324737549 | 0.1484441251212925 | 0.14456711157737018 | 0.15370444473024028 |
| 07/09/2024, 11:15:18 | ./Teamim-tiny_DropOut-0.5_New-Data_date-09-07-2024_11-13 | 8 | 1 | 1e-05 | 10 | 500 | True | {'use_reentrant': False} | True | steps | 32 | True | 225 | 500 | 500 | 1 | ['tensorboard'] | True | avg_f1_Exact | True | True | 1.077277862995863 | 2.0161290322580645 | 500 | 1.3493123054504395 | 0.11196754871749044 | 0.10726905771306977 | 0.11832617585992708 |
| 07/09/2024, 11:36:15 | ./Teamim-tiny_Old-Data_date-09-07-2024_11-28 | 8 | 1 | 1e-05 | 10 | 5000 | True | {'use_reentrant': False} | True | steps | 32 | True | None | 5000 | 5000 | 25 | ['tensorboard'] | True | avg_f1_Exact | True | True | 0.5126914912223816 | 0.5728689275893676 | 5000 | 0.3567802906036377 | 0.634995933585646 | 0.6379584155289065 | 0.6338327879650193 |
| 07/09/2024, 11:42:25 | ./Teamim-tiny_Old-Data_date-09-07-2024_11-35 | 8 | 1 | 1e-05 | 10 | 5000 | True | {'use_reentrant': False} | True | steps | 32 | True | None | 5000 | 5000 | 1 | ['tensorboard'] | True | avg_f1_Exact | True | True | 0.18801606579720975 | 0.5728689275893676 | 5000 | 0.31880703568458557 | 0.6706442946707739 | 0.6743758098704623 | 0.668774170427063 |
| 07/09/2024, 11:53:38 | ./Teamim-tiny_Old-Data_date-09-07-2024_11-49 | 8 | 1 | 1e-05 | 10 | 2000 | True | {'use_reentrant': False} | True | steps | 32 | True | None | 2000 | 2000 | 1 | ['tensorboard'] | True | avg_f1_Exact | True | True | 0.8328142464011907 | 0.229147571035747 | 2000 | 0.5039238333702087 | 0.4936389928158449 | 0.4886240458797581 | 0.5012416646714167 |
| 07/09/2024, 11:58:28 | ./Teamim-tiny_Old-Data_date-09-07-2024_11-54 | 8 | 1 | 1e-05 | 10 | 2000 | True | {'use_reentrant': False} | True | steps | 32 | True | None | 2000 | 2000 | 1 | ['tensorboard'] | True | avg_f1_Exact | True | True | 0.365867255397141 | 0.229147571035747 | 2000 | 0.3993128836154938 | 0.5797832830507968 | 0.5774234055165023 | 0.584343533723699 |
| 07/09/2024, 12:09:02 | ./Teamim-tiny_DropOut-0.5_Old-Data_date-09-07-2024_12-05 | 8 | 1 | 1e-05 | 10 | 2000 | True | {'use_reentrant': False} | True | steps | 32 | True | None | 2000 | 2000 | 25 | ['tensorboard'] | True | avg_f1_Exact | True | True | 0.8246453580856323 | 0.229147571035747 | 2000 | 0.4964655935764313 | 0.47605439414429945 | 0.475031230810278 | 0.47864142285629896 |
| 07/09/2024, 12:15:58 | ./Teamim-tiny_DropOut-0.5_Old-Data_date-09-07-2024_12-10 | 8 | 1 | 1e-05 | 10 | 2000 | True | {'use_reentrant': False} | True | steps | 32 | True | None | 2000 | 2000 | 25 | ['tensorboard'] | True | avg_f1_Exact | True | True | 5.292523445129395 | 0.229147571035747 | 2000 | 4.263174533843994 | 0.0 | 0.0 | 0.0 |
| 07/09/2024, 12:23:25 | ./Teamim-tiny_Old-Data_date-09-07-2024_12-19 | 8 | 1 | 1e-05 | 10 | 2000 | True | {'use_reentrant': False} | True | steps | 32 | True | None | 2000 | 2000 | 25 | ['tensorboard'] | True | avg_f1_Exact | True | True | 0.8288912086486816 | 0.229147571035747 | 2000 | 0.4990358352661133 | 0.4843666288922216 | 0.48244298149499903 | 0.4881738287730023 |
| 07/09/2024, 12:29:29 | ./Teamim-tiny_Old-Data_date-09-07-2024_12-25 | 8 | 1 | 1e-05 | 10 | 2000 | True | {'use_reentrant': False} | True | steps | 32 | True | None | 2000 | 2000 | 25 | ['tensorboard'] | True | avg_f1_Exact | True | True | 0.3652078564167023 | 0.229147571035747 | 2000 | 0.3952443301677704 | 0.5750990925954228 | 0.5760319179011449 | 0.5761704243109201 |
| 07/09/2024, 16:27:34 | ./Teamim-large-v2_WeightDecay-0.001_Augmented_Combined-Data_date-09-07-2024_12-45 | 8 | 1 | 1e-05 | 10 | 3000 | True | {'use_reentrant': False} | True | steps | 32 | True | None | 1000 | 1000 | 25 | ['tensorboard'] | True | avg_f1_Exact | True | True | 0.18225202290217082 | 0.15500671695773482 | 3000 | 0.10876211524009705 | 0.8742152068160314 | 0.874028143812048 | 0.8756704628054444 |
| 07/09/2024, 20:26:39 | ./Teamim-large-v2_WeightDecay-0.05_Augmented_Combined-Data_date-09-07-2024_16-25 | 8 | 1 | 1e-05 | 10 | 3000 | True | {'use_reentrant': False} | True | steps | 32 | True | None | 1000 | 1000 | 25 | ['tensorboard'] | True | avg_f1_Exact | True | True | 0.18418040355046592 | 0.15500671695773482 | 3000 | 0.10779104381799698 | 0.8753049831557934 | 0.8757985810756497 | 0.8760230096430711 |
| 07/09/2024, 20:28:27 | ./Teamim-tiny_WeightDecay-0.05_Augmented_Old-Data_date-09-07-2024_20-24 | 8 | 1 | 1e-05 | 10 | 100 | True | {'use_reentrant': False} | True | steps | 32 | True | None | 100 | 100 | 1 | ['tensorboard'] | True | avg_f1_Exact | True | True | 3.929404218196869 | 0.011457378551787351 | 100 | 2.541598081588745 | 0.010927425159273266 | 0.011082292050566352 | 0.011376425839235756 |
| 07/10/2024, 02:15:49 | ./Teamim-large-v2_WeightDecay-0.1_Augmented_Combined-Data_date-09-07-2024_22-34 | 8 | 1 | 1e-05 | 10 | 3000 | True | {'use_reentrant': False} | True | steps | 32 | True | None | 1000 | 1000 | 25 | ['tensorboard'] | True | avg_f1_Exact | True | True | 0.181333984653155 | 0.15500671695773482 | 3000 | 0.10994978994131088 | 0.8625706975697263 | 0.8618330635588181 | 0.8645391287123869 |
| 07/10/2024, 10:50:00 | ./Teamim-tiny_WeightDecay-0.05_Augmented_New-Data_date-10-07-2024_10-45 | 32 | 1 | 1e-05 | 10 | 200 | True | {'use_reentrant': False} | True | steps | 32 | True | None | 200 | 200 | 1 | ['tensorboard'] | True | avg_f1_Exact | True | True | 5.8823949813842775 | 3.225806451612903 | 200 | 4.498262882232666 | 0.00515324005770875 | 0.00771854131152231 | 0.007464007685499971 |
| 07/10/2024, 10:57:30 | ./Teamim-tiny_WeightDecay-0.05_Augmented_New-Data_date-10-07-2024_10-55 | 32 | 1 | 1e-05 | 10 | 200 | True | {'use_reentrant': False} | True | steps | 32 | True | None | 200 | 200 | 1 | ['tensorboard'] | True | avg_f1_Exact | True | True | 5.933233528137207 | 3.225806451612903 | 200 | 4.567381858825684 | 0.015275305979308354 | 0.01370306340988145 | 0.01776123605391898 |
| 07/10/2024, 11:02:02 | ./Teamim-tiny_WeightDecay-0.05_Augmented_New-Data_date-10-07-2024_11-01 | 32 | 1 | 1e-05 | 10 | 20 | True | {'use_reentrant': False} | True | steps | 32 | True | None | 20 | 20 | 1 | ['tensorboard'] | True | avg_f1_Exact | True | True | 12.212247085571288 | 0.3225806451612903 | 20 | 10.23202133178711 | 0.0 | 0.0 | 0.0 |
| 07/10/2024, 11:06:12 | ./Teamim-tiny_WeightDecay-0.05_Augmented_New-Data_date-10-07-2024_11-05 | 32 | 1 | 1e-05 | 10 | 20 | True | {'use_reentrant': False} | True | steps | 32 | True | None | 20 | 20 | 1 | ['tensorboard'] | True | avg_f1_Exact | True | True | 11.964767503738404 | 0.3225806451612903 | 20 | 10.37028980255127 | 0.0 | 0.0 | 0.0 |
| 07/10/2024, 11:13:09 | ./Teamim-tiny_WeightDecay-0.05_Augmented_New-Data_date-10-07-2024_11-12 | 32 | 1 | 1e-05 | 10 | 20 | True | {'use_reentrant': False} | True | steps | 32 | True | None | 20 | 20 | 1 | ['tensorboard'] | True | avg_f1_Exact | True | True | 12.033249139785767 | 0.3225806451612903 | 20 | 10.375862121582031 | 0.0 | 0.0 | 0.0 |
| 07/10/2024, 11:16:09 | ./Teamim-tiny_WeightDecay-0.05_Augmented_New-Data_date-10-07-2024_11-15 | 32 | 1 | 1e-05 | 10 | 20 | True | {'use_reentrant': False} | True | steps | 32 | True | 225 | 20 | 20 | 1 | ['tensorboard'] | True | avg_f1_Exact | True | True | 12.690243005752563 | 0.3225806451612903 | 20 | 10.641993522644043 | 0.0 | 0.0 | 0.0 |
| 07/10/2024, 11:19:33 | ./Teamim-tiny_WeightDecay-0.05_Augmented_New-Data_date-10-07-2024_11-18 | 32 | 1 | 1e-05 | 10 | 20 | True | {'use_reentrant': False} | True | steps | 32 | True | 225 | 20 | 20 | 1 | ['tensorboard'] | True | avg_f1_Exact | True | True | 8.46168749332428 | 0.3225806451612903 | 20 | 5.885577201843262 | 0.0005421523448088912 | 0.00030432136335970786 | 0.0024813895781637717 |


| Model Name | Model Name | data | steps | lr |
|----------|----------|----------|--------|--------|
| whisper-medium-he-teamim-base | medium | all | 10,000 | 3e-5 |
| whisper-medium-he-teamim-ashkenazi-01 | base | ashkenazi | 9,000 | 1e-6 |




# Test the model

In [35]:
model_name = "cantillation/Teamim-large-v2_Random-True_Augmented_date-26-06-2024_10-34-39"

In [36]:
# load the model
model = WhisperForConditionalGeneration.from_pretrained(model_name)
processor = WhisperProcessor.from_pretrained(model_name)

# load the test data
# test_data = parashat_hashavua_dataset(few_data=FASTTEST, train=False ,validation=False, test=True,  num_of_words_in_sample=4, nusachim=NUSACHIM, processor=processor)

# create the data collator
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)


Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  2.15it/s]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [37]:
from transformers import Seq2SeqTrainer
trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    eval_dataset=test_data,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

# evaluate the model
results = trainer.evaluate() # we use evaluate to get the metrics
print(results)
# save the results to a json file
# create the results file
with open(f"results_{MODEL_NAME.split('/')[-1]}.json", 'w') as f:
    json.dump(results, f, indent=4)


NameError: name 'test_data' is not defined

In [ ]:
# that was just on small test data, we need to evaluate on a bigger test data in the future !!!

# The model that was trained only on one dataset (which is slightly more similar to the style of the test data)
{'eval_loss': 0.7734131813049316, 'eval_wer': 61.811023622047244, 'eval_avg_precision_Exact': 0.46710057318055453, 'eval_avg_recall_Exact': 0.5216817130186114, 'eval_avg_f1_Exact': 0.4904648408291106, 'eval_avg_precision_Letter_Shift': 0.4789416732569487, 'eval_avg_recall_Letter_Shift': 0.5334662248031231, 'eval_avg_f1_Letter_Shift': 0.5022712162718498, 'eval_avg_precision_Word_Level': 0.500396535917345, 'eval_avg_recall_Word_Level': 0.5445642177192979, 'eval_avg_f1_Word_Level': 0.518676288124462, 'eval_avg_precision_Word_Shift': 0.5926319891153651, 'eval_avg_recall_Word_Shift': 0.6415232658013406, 'eval_avg_f1_Word_Shift': 0.6130332240673503, 'eval_precision_median_exact': 0.5185185185185185, 'eval_recall_median_exact': 0.5714285714285714, 'eval_f1_median_exact': 0.5384615384615384, 'eval_precision_max_exact': 0.7857142857142857, 'eval_recall_max_exact': 0.8181818181818182, 'eval_f1_max_exact': 0.8, 'eval_precision_min_Exact': 0.0, 'eval_recall_min_Exact': 0.0, 'eval_f1_min_Exact': 0.0, 'eval_precision_min_Letter_Shift': 0.0, 'eval_recall_min_Letter_Shift': 0.0, 'eval_f1_min_Letter_Shift': 0.0, 'eval_precision_min_Word_Level': 0.0, 'eval_recall_min_Word_Level': 0.0, 'eval_f1_min_Word_Level': 0.0, 'eval_precision_min_Word_Shift': 0.25, 'eval_recall_min_Word_Shift': 0.2777777777777778, 'eval_f1_min_Word_Shift': 0.2631578947368421}

# The model that was trained on two datasets (the second dataset is much more different from the test data)
{'eval_loss': 1.015082836151123, 'eval_wer': 61.811023622047244, 'eval_avg_precision_Exact': 0.49305580535527055, 'eval_avg_recall_Exact': 0.4932712533026013, 'eval_avg_f1_Exact': 0.49279243354847346, 'eval_avg_precision_Letter_Shift': 0.4963025586020238, 'eval_avg_recall_Letter_Shift': 0.4964060495408457, 'eval_avg_f1_Letter_Shift': 0.49598222621195037, 'eval_avg_precision_Word_Level': 0.5137405627001068, 'eval_avg_recall_Word_Level': 0.5019255300603264, 'eval_avg_f1_Word_Level': 0.5073072310311691, 'eval_avg_precision_Word_Shift': 0.6237927402365905, 'eval_avg_recall_Word_Shift': 0.613736202842786, 'eval_avg_f1_Word_Shift': 0.6182032164583927, 'eval_precision_median_exact': 0.5294117647058824, 'eval_recall_median_exact': 0.5416666666666666, 'eval_f1_median_exact': 0.5142857142857143, 'eval_precision_max_exact': 0.7142857142857143, 'eval_recall_max_exact': 0.6818181818181818, 'eval_f1_max_exact': 0.6976744186046512, 'eval_precision_min_Exact': 0.05, 'eval_recall_min_Exact': 0.05, 'eval_f1_min_Exact': 0.05000000000000001, 'eval_precision_min_Letter_Shift': 0.05, 'eval_recall_min_Letter_Shift': 0.05, 'eval_f1_min_Letter_Shift': 0.05000000000000001, 'eval_precision_min_Word_Level': 0.05, 'eval_recall_min_Word_Level': 0.05, 'eval_f1_min_Word_Level': 0.05000000000000001, 'eval_precision_min_Word_Shift': 0.3, 'eval_recall_min_Word_Shift': 0.3, 'eval_f1_min_Word_Shift': 0.3}

# we can see that the model that was trained on two datasets has a higher f1 score